In [25]:
import note_seq
from magenta.models.drums_rnn import drums_rnn_sequence_generator
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import music_pb2, generator_pb2
import tensorflow as tf

bundle = sequence_generator_bundle.read_bundle_file('/app/models/drum_kit_rnn/drum_kit_rnn.mag')
generator_map = drums_rnn_sequence_generator.get_generator_map()
drums_rnn = generator_map['drum_kit'](checkpoint=None, bundle=bundle)
drums_rnn.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpa2kwjabl/model.ckpt


In [24]:
# Empty 1-bar sequence
seed = music_pb2.NoteSequence()
seed.ticks_per_quarter = 220
note_seq.sequence_proto_to_midi_file(seed, 'drum_01_seed.mid')

generator_options = generator_pb2.GeneratorOptions()
generator_options.generate_sections.add(start_time=seed.total_time, end_time=4.0)  # generate 4 bars

drum_seq = drums_rnn.generate(seed, generator_options)
note_seq.sequence_proto_to_midi_file(drum_seq, 'drum_01_gen.mid')

INFO:tensorflow:Beam search yields sequence with log-likelihood: -23.706131 


In [23]:
# Funky drums
seed = music_pb2.NoteSequence()
seed.ticks_per_quarter = 220  # standard

# Kick drum (MIDI 36)
seed.notes.add(pitch=36, start_time=0.0, end_time=0.25, velocity=100)
seed.notes.add(pitch=36, start_time=0.75, end_time=1.0, velocity=100)

# Snare drum (MIDI 38)
seed.notes.add(pitch=38, start_time=0.5, end_time=0.75, velocity=110)
seed.notes.add(pitch=38, start_time=1.5, end_time=1.75, velocity=110)

# Hi-hats (MIDI 42)
seed.notes.add(pitch=42, start_time=0.25, end_time=0.5, velocity=70)
seed.notes.add(pitch=42, start_time=0.75, end_time=1.0, velocity=80)
seed.notes.add(pitch=42, start_time=1.25, end_time=1.5, velocity=70)
seed.notes.add(pitch=42, start_time=1.75, end_time=2.0, velocity=80)

seed.total_time = 2.0

note_seq.sequence_proto_to_midi_file(seed, 'drum_02_seed.mid')

generator_options = generator_pb2.GeneratorOptions()
section = generator_options.generate_sections.add(
    start_time=seed.total_time,
    end_time=6.0  # generates ~4 more bars
)
generator_options.args['temperature'].float_value = 1.0  # higher = more variation

drum_seq = drums_rnn.generate(seed, generator_options)
note_seq.sequence_proto_to_midi_file(drum_seq, 'drum_02_gen.mid')

INFO:tensorflow:Beam search yields sequence with log-likelihood: -13.984137 


In [29]:
bundle = sequence_generator_bundle.read_bundle_file('/app/models/basic_rnn/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpc4fpio6z/model.ckpt


In [32]:
# simple 4 note seq
seed = music_pb2.NoteSequence()
seed.ticks_per_quarter = 220

# C4, D4, E4, G4 (quarter notes)
seed.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
seed.notes.add(pitch=62, start_time=0.5, end_time=1.0, velocity=80)
seed.notes.add(pitch=64, start_time=1.0, end_time=1.5, velocity=80)
seed.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)

seed.total_time = 2.0
note_seq.sequence_proto_to_midi_file(seed, 'melody_01_seed.mid')

generator_options = generator_pb2.GeneratorOptions()
generator_options.generate_sections.add(
    start_time=seed.total_time,
    end_time=10.0  # generate 10 more seconds
)
generator_options.args['temperature'].float_value = 1.0  # higher = more creative

generated_sequence = melody_rnn.generate(seed, generator_options)
note_seq.sequence_proto_to_midi_file(generated_sequence, 'melody_01_gen.mid')

INFO:tensorflow:Beam search yields sequence with log-likelihood: -40.940739 
